In [1]:
# necessary libraries
import numpy as np
import pandas as pd
import re
import time

# selenium web scrapping 
from selenium import webdriver 
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

# Testing Selenium for login and searching Google

Logging in google accounts via selenium is not working:

https://gist.github.com/ikegami-yukino/51b247080976cb41fe93#

ERROR: "This browser or app may not be secure"


# Functions:

In [40]:
# NOT WORKING at all
# def login_google(email,password):
#     url = "https://accounts.google.com/ServiceLogin?service=mail&continue=https://mail.google.com/mail/#identifier"
#     browser.get(url)
#     browser.find_element_by_xpath('//*[@id="identifierId"]').send_keys(email + Keys.ENTER)
#     WebDriverWait(browser, 10).until(
#         EC.presence_of_element_located((By.XPATH, '//*[@id="password"]/div[1]/div/div[1]/input'))).send_keys(password + Keys.ENTER)
    

In [2]:
def ChangeProxy(proxy):
    capabilities = webdriver.DesiredCapabilities.CHROME.copy()
    capabilities['proxy'] = {
        "httpProxy": proxy,
        "ftpProxy": proxy,
        "sslProxy": proxy,
        "noProxy": None,
        "proxyType": "MANUAL",
        "class": "org.openqa.selenium.Proxy",
        "autodetect": False
    }
    
    return(capabilities)

In [3]:
# WORKING but Google blocks account in a while
def LoginGoogle(email, password):
    # new driver for a new account (signed-in search)
    browser = webdriver.Chrome(ChromeDriverManager().install(), desired_capabilities=None)

    # log in using stackoverflow.com 
    #browser.get('https://accounts.google.com/signin/oauth/identifier?client_id=717762328687-iludtf96g1hinl76e4lc1b9a82g457nn.apps.googleusercontent.com&as=hhP6IOeJDYTyWGPFxmNFCw&destination=https%3A%2F%2Fstackauth.com&approval_state=!ChRfbmZaSWxMWXFuNVBGQW0wUWFFNBIfVXhRVnRaSk0taFlkMEVBN1JaNXdOM09OS3RIREJ4Yw%E2%88%99AF-3PDcAAAAAXlZfFoFshMv4nBqo0_jkZo9RQXsT6UO-&oauthgdpr=1&xsrfsig=ChkAeAh8Tz0dCH3vnLQbQ9X-nhFhFN76cDt_Eg5hcHByb3ZhbF9zdGF0ZRILZGVzdGluYXRpb24SBXNvYWN1Eg9vYXV0aHJpc2t5c2NvcGU&flowName=GeneralOAuthFlow')
    
    # or soundcloud.com
    browser.get('https://accounts.google.com/signin/oauth/identifier?client_id=984739005367.apps.googleusercontent.com&as=OjPXjsHQbOoh7aJEF5MxLQ&destination=https%3A%2F%2Fsoundcloud.com&approval_state=!ChR3eHVpME5ETDVySFpIOTRyRnNfWBIfazZySHhzcTdlaW9XMEVBN1JaNXdOM1BOaEozREJ4Yw%E2%88%99AF-3PDcAAAAAXlZeQqSVcjiMkVRNnD0fNd6-Yooea5Eu&oauthgdpr=1&xsrfsig=ChkAeAh8TxrMJywUUOdyRWIQPx0KoVZpLMOBEg5hcHByb3ZhbF9zdGF0ZRILZGVzdGluYXRpb24SBXNvYWN1Eg9vYXV0aHJpc2t5c2NvcGU&flowName=GeneralOAuthFlow')

    time.sleep(3)
    browser.find_element_by_xpath('//*[@id="identifierId"]').send_keys(email + Keys.ENTER) 
    time.sleep(3)
    browser.find_element_by_xpath('//input[@type="password"]').send_keys(password + Keys.ENTER)
    
    return(browser)

In [4]:
def LogoutGoogle():
    # new driver for signed-out search
    browser = webdriver.Chrome(ChromeDriverManager().install(), desired_capabilities=None)

    browser.get('https://www.google.com/history/optout?cb=1&hl=en&continue=https%3A%2F%2Fconsent.google.com%2Fui%2F%3Fcontinue%3Dhttps%3A%2F%2Fwww.google.com%2Fsearch%3Fsource%253Dhp%2526ei%253D-Q9VXriOKoHeas2YofAO%2526q%253DSoftware%252Btesting%26origin%3Dhttps%3A%2F%2Fwww.google.com%26if%3D1%26wp%3D71%26gl%3DFR%26hl%3Den%26pc%3Ds%23controls')
    browser.find_element_by_xpath('/html/body/div/div[1]/div[2]/div/table/tbody/tr/td[2]/form/button/div').click()
    
    return(browser)

In [29]:
def SearchGoogle(query, browser, max_num_res=10):
    
    browser.get("https://google.com") 
    browser.find_element_by_xpath("//input[@name='q']").send_keys(query + Keys.ENTER)
    
    search_results = []
    elements = browser.find_elements_by_class_name('r')
    # scrapping first 'max_num_res' results (or less)
    for i in range(len(elements)): 
        if i==max_num_res:
            break
        search_results.append( elements[i].find_element_by_css_selector('a').get_attribute('href') )
        
    return search_results
    

# Main:

In [ ]:
# install Google Chrome driver and use localhost 
# browser = webdriver.Chrome(ChromeDriverManager().install(), desired_capabilities=change_proxy(proxy = "localhost:8080"))

In [ ]:
# browser.save_screenshot('screenshot.png')

### Logged in

In [35]:
%%capture
# logged-in session 
# account: sex:men, age:43, ip:FR, purpose:testing selenium 
browser_logged_in = LoginGoogle(email='alexfromensk@gmail.com',
                                 password='Ensk2capitana') 

In [68]:
# signed-in search for a particular query
results_logged_in = SearchGoogle(query="sport",
             browser=browser_logged_in,
             max_num_res=10)

In [69]:
results_logged_in

['https://www.sport.fr/',
 'https://www.sports.fr/',
 'https://www.lequipe.fr/',
 'http://sport24.lefigaro.fr/',
 'https://en.wikipedia.org/wiki/Sport',
 'https://fr.wikipedia.org/wiki/Sport',
 'https://www.20minutes.fr/sport/',
 'https://sports.orange.fr/']

### Logged out 

In [45]:
%%capture
# logged-out session
browser_logged_out = LogoutGoogle() 

In [66]:
# ssigned-out search for a particular query
results_logged_out = SearchGoogle(query="sport",
             browser=browser_logged_out,
             max_num_res=10)

In [67]:
results_logged_out

['https://www.sport.fr/',
 'https://www.sports.fr/',
 'https://www.lequipe.fr/',
 'http://sport24.lefigaro.fr/',
 'https://en.wikipedia.org/wiki/Sport',
 'https://fr.wikipedia.org/wiki/Sport',
 'https://www.20minutes.fr/sport/',
 'https://sports.orange.fr/',
 'https://www.lemonde.fr/sport/']

### Analysis

In [70]:
# show differences between two search results
set(results_logged_out).symmetric_difference(results_logged_in)

{'https://www.lemonde.fr/sport/'}

In [ ]:
# show similarities between two search results
# set(results_logged_out).intersection(results_logged_in)